# Concours MTH3302
#### Polytechnique Montréal
### Projet A2024
----
### Objectif
Prédire **la `consommation` en carburant de voitures récentes**.

### Données
Le jeu de données contient pour presque 400 véhicule, la consommation moyenne en L/100km, l'année de frabrication, le type de véhicule, le nombre de cylindre, cylindree, la transmission et la boite.

- `train.csv` est l'ensemble d'entraînement
- `test.csv` est l'ensemble de test


## Chargement des données

Importation des librairies utilisées dans le calepin.

In [1]:
using CSV, DataFrames, Statistics, Dates, Gadfly, Combinatorics, Random, LinearAlgebra

Premier fichier est l'ensemble des données pour l'entrainement, il contient l'année, le type, le nombre_cylindres, la cylindree, la transmission, la boite, la consommation.

In [5]:
trainData = CSV.read("./data/train.csv", DataFrame)
trainData.consommation = parse.(Float64,replace.(trainData.consommation, "," => "."))
trainData.cylindree = parse.(Float64,replace.(trainData.cylindree, "," => "."))
trainData[!, :id] = 1:nrow(trainData)
first(trainData, 1)

Row,annee,type,nombre_cylindres,cylindree,transmission,boite,consommation,id
,Int64,String31,Int64,Float64,String15,String15,Float64,Int64
1,2023,voiture_moyenne,8,4.4,integrale,automatique,13.8359,1


Le deuxième fichier est l'ensemble des données pour le test, il contient l'année, le type, le nombre_cylindres, la cylindree, la transmission, la boite. Ici il n'a pas la consommation car il s'agit de la variable d'intérêt.

In [ ]:
testData = CSV.read("./data/test.csv", DataFrame)
first(testData, 1)

## Exploration des données

Lors de l’étape d’exploration des données, notre objectif principal était de mieux comprendre les caractéristiques disponibles afin d’évaluer leur pertinence pour la prédiction de la `consommation` en carburant. Nous avons adopté une approche intuitive, en nous appuyant sur nos connaissances de base concernant les véhicules et leur consommation, pour repérer les tendances et des relations potentielles entre les variables.

#### Chargement donnée pour l'exploration

Chargeons les données pour l'exploration

In [ ]:
trainDataExploration = CSV.read("./data/train.csv", DataFrame)
testDataExploration = CSV.read("./data/test.csv", DataFrame)
first(trainDataExploration, 1)

### Données manquantes

Vérifions s'il y a des données manquantes dans l'ensemble d'entrainement et de test.

In [1]:
function missing_data(data)
    return  mapcols(x -> sum(ismissing.(x)), data)
end

missing_data (generic function with 1 method)

In [ ]:
missing_data(trainDataExploration)

In [ ]:
missing_data(testDataExploration)

Nous pouvons constater qu'il n'y a pas de données manquantes dans les données fournies.

### Conversion des données : préparation des colonnes pour l'analyse et l'évaluation

Étant donné que les colonnes `cylindree` et `consommation` sont actuellement définies comme `String3` et `String31` respectivement, il est nécessaire de modifier leur type en `Float64` dans les deux ensembles de données afin de pouvoir les analyser correctement.

In [ ]:
trainDataExploration.consommation = parse.(Float64,replace.(trainDataExploration.consommation, "," => "."))
trainDataExploration.cylindree = parse.(Float64,replace.(trainDataExploration.cylindree, "," => "."))
first(trainDataExploration, 1)

### Création de nouvelles variables explicatives : 

Pour enrichir notre analyse et améliorer les performances du modèle, nous avons ajouté de nouvelles variables explicatives.  Il est a noter qu'il se peut qu'au final on ne les utilise pas. Nous ferons une synthèse des variables explicatives sélectionnées après l'exploration des données.

#### `volume_gaz`
Cette variable est calculée comme le produit du nombre de cylindres et de la cylindrée du moteur. Elle reflète la capacité totale de déplacement du moteur, ce qui pourrait avoir une influence directe sur la consommation de carburant.

In [ ]:
trainDataExploration[!,:volume_gaz] = trainDataExploration[!,:nombre_cylindres] .* trainDataExploration[!,:cylindree]
first(trainDataExploration, 1)

#### `weight`
Nous avons regarder le poids moyen par types de véhicules en se fiant aux liens suivants:
https://www.insurancenavy.com/average-car-weight/
https://www.auto-tests.com/fr/lightest-weight/Wagon/all/. L'ajout de cette variable a pour but d'établir un lien entre les différents types de véhicules.

In [ ]:
weight_dict = Dict(
    "voiture_moyenne" => 3300, 
    "VUS_petit" => 3500, 
    "voiture_compacte" => 2800, 
    "voiture_deux_places" => 2800, 
    "voiture_minicompacte" => 1500, 
    "VUS_standard" => 5000, 
    "monospace" => 4500, 
    "voiture_sous_compacte" => 2600, 
    "camionnette_petit" => 4200, 
    "break_petit" => 2640, 
    "voiture_grande" => 4400, 
    "camionnette_standard" => 4700, 
    "break_moyen" => 3300)
trainDataExploration[!, :weight] = [weight_dict[t] for t in trainDataExploration[!, :type]]
first(trainDataExploration, 1)


### Recherche de relation entre les variables explicatives et la variable d'intérêt

Analysons maintenant chacune des variables de l'ensemble d'entraînement en fonction de notre variable cible, la `consommation`. Cette étape vise à détecter les relations potentielles entre les variables. Concrètement, nous traçons des graphiques de `consommation` en fonction de différentes variables, telles que : `annee`, `type`, `nombre_cylindres`, `cylindree`, `transmission`, `boite`, `volume_gaz` et `weight`. Nous faisons cela afin de détecter une tendance.

In [ ]:
variables = [:annee, :type, :nombre_cylindres, :cylindree, :transmission, :boite, :volume_gaz, :weight]

plots = [
    Gadfly.plot(
        trainDataExploration,
        x=var,
        y=:consommation,
        Geom.point,
        Guide.xlabel(string(var)),
        Guide.ylabel("consommation")
    ) for var in variables
]

set_default_plot_size(35cm, 35cm)
p = reshape(plots, (4,2))
gridstack(p)


In [ ]:
valeurs_nombre_cylindres = unique(trainDataExploration[:,:nombre_cylindres])
println("Valeurs distinctes de la colonne :nombre_cylindres : ", valeurs_nombre_cylindres)

Analyse des graphiques ci-dessus:

Pour les variables explicatives suivantes en fonction de la consommation:
- `annee` : La distribution de la consommation selon les années semble relativement homogène. On observe cependant une légère diminution de la consommation en 2020 et 2021, probablement attribuable aux effets de la pandémie. Par conséquent, il apparaît que cette variable n'entretient pas de relation linéaire évidente avec la consommation. Cette variable n'aura donc probablement pas d'influence sur la consommation.

- `type`: Pour certains types de véhicules, la quantité de données disponibles est insuffisante, ce qui complique une évaluation fiable et précise de leur impact sur la consommation. Cette limitation justifie la nécessité de mener une analyse complémentaire dédiée à cette variable, comme détaillé dans la prochaine section intitulée "Analyse par type de véhicule". 

- `nombre_cylindres`: On observe que cette variable peut prendre 7 valeurs distinctes : [3, 4, 5, 6, 8, 10, 12]. Parmi celles-ci, trois valeurs (5, 10 et 12) ont chacune très peu de données. De plus, on remarque une tendance où l'augmentation du nombre de cylindres est associée à une augmentation de la consommation, suggérant une possible relation linéaire entre ces deux variables. 

- `cylindree` : On remarque une tendance où l'augmentation de la cylindree est associée à une augmentation de la consommation, suggérant une possible relation linéaire entre ces deux variables.

- `transmission`: Les transmissions intégrale, propulsion et 4x4 présentent une distribution de consommation relativement similaire. En revanche, la transmission traction semble afficher une tendance à une consommation inférieure par rapport aux autres.

- `boite`: Le type de boîte de vitesses ne semble pas influencer significativement la consommation, car la distribution de la consommation pour les boîtes automatiques est globalement similaire à celle des boîtes manuelles.

- `volume_gaz` : La relation entre les deux variables semble linéaire ou même semble logarithmique. Cependant, on remarque qu'il y a beaucoup plus de données pour des volumes faibles que pour des volumes élevés.

- `weight`: Étant donné que le poids prend en compte les différents types de véhicules, il est difficile d'analyser une relation directe entre le poids et la consommation. Malgré ça, pour le moment on ne voit pas de relation linéaire entre les deux variables.


### Analyse par `type` de véhicule

Nous poursuivons l'exploration en examinant les données par `type`. Cette étape vise à identifier des comportements spécifiques à chaque `type`. En regroupant les véhicules par transmission au sein de chaque `type`, nous calculons la `consommation` moyenne, le `volume_gaz` moyen et le nombre d'observations dans chaque groupe. Cela nous permet de mieux comprendre comment ces facteurs interagissent pour chaque `type` de véhicule et d'évaluer si le `type` ou la `transmission` influence significativement la `consommation`. Cette analyse guide ainsi la sélection des variables pertinentes pour le modèle.

In [ ]:
for type in unique(trainDataExploration.type)
    println(type)
    data_type = trainDataExploration[trainDataExploration.type .== type, :]
    println(combine(groupby(data_type, :transmission), :consommation => mean, :volume_gaz => mean, nrow => :nrow))
    println()
end

Dans les tableaux précédents, nous constatons que, pour une voiture moyenne, une voiture compacte, une voiture minicompacte et une voiture_sous_compacte avec une transmission par traction, la consommation moyenne et le volume de gaz moyen sont plus faibles que pour les autres types de transmission. Cependant, pour les autres types de véhicules, ces valeurs restent similaires, quelle que soit la transmission. Ainsi, cette observation ne semble pas apporter de conclusions claires quant à une relation significative.

Nous passons maintenant à une visualisation plus détaillée. Pour chaque `type` de véhicule, nous traçons la relation entre le `volume_gaz` et la `consommation`. Cette étape vise à observer s'il existe des tendances ou des corrélations spécifiques entre ces deux variables pour chaque `type`. Ces visualisations permettent de détecter des comportements distincts ou des patrons dans les données, ce qui peut orienter la sélection des variables explicatives ou révéler des transformations nécessaires pour améliorer la précision du modèle.

In [ ]:
plots = []

for type in unique(trainDataExploration.type)
    data_type = trainDataExploration[trainDataExploration.type .== type, :]
    push!(plots, plot(
        x=data_type.volume_gaz,
        y=data_type.consommation,
        Geom.point,
        Guide.title("Type: $type"),
        Guide.xlabel("Volume Gaz"),
        Guide.ylabel("Consommation")
    ))
end

set_default_plot_size(10cm, 10cm)

for i in 1:length(plots)
    display(plots[i])
end 

Étant donné qu'il y a très peu de données pour certains `type`, il est difficile d'analyser et d'identifier des tendances dans les graphiques ci-dessus.

#### Ajout variable : `general_type`

L'ajout de la variable `general_type` dans les données d'exploration permet de regrouper les types spécifiques de véhicules en catégories plus générales telles que "voiture", "VUS", "camionnette" ou "break". Ces catégories sont basées sur le type de carrosserie de la voiture. Cela simplifie l'analyse en réduisant la granularité de la variable `type`, qui contient de nombreuses sous-catégories. En utilisant ces regroupements, on peut mieux voir les tendances globales et réduire la complexité du modèle de prédiction, tout en conservant les différences majeures entre les groupes de véhicules. 

In [ ]:
general_type_dict = Dict(
    "voiture_moyenne" => "voiture", 
    "VUS_petit" => "VUS", 
    "voiture_compacte" => "voiture", 
    "voiture_deux_places" => "voiture", 
    "voiture_minicompacte" => "voiture", 
    "VUS_standard" => "VUS", 
    "monospace" => "camionnette", 
    "voiture_sous_compacte" => "voiture", 
    "camionnette_petit" => "camionnette", 
    "break_petit" => "break", 
    "voiture_grande" => "voiture", 
    "camionnette_standard" => "camionnette", 
    "break_moyen" => "break")

trainDataExploration[!, :general_type] = [general_type_dict[t] for t in trainDataExploration[!, :type]]
unique(trainDataExploration[!, :general_type])

Examinons maintenant le `volume_gaz` par rapport à la `consommation` pour chacun des `general_type`.

In [ ]:
plots = []

for general_type in unique(trainDataExploration.general_type)
    data_type = trainDataExploration[trainDataExploration.general_type .== general_type, :]
    push!(plots, plot(
        x=data_type.volume_gaz,
        y=data_type.consommation,
        Geom.point,
        Guide.title("General Type: $general_type"),
        Guide.xlabel("Volume Gaz"),
        Guide.ylabel("Consommation")
    ))
end

set_default_plot_size(10cm, 10cm)

for i in 1:length(plots)
    display(plots[i])
end

Les graphiques ci-dessus semble montrer une tendance linéaire, où une augmentation du volume de gaz (x) semble être associée à une hausse de la consommation (y). Ainsi, la variable `general_type` pourrait être une variable explicative pertinente pour comprendre la consommation.

### Transformation des variables explicatives: inclusion des variables qualitatives

L'encodage des variables explicatives qualitatives `general_type`, `transmission` et `boite` est essentiel afin d'être en mesure de les transformer en format numérique et de pouvoir les intégrer dans nos modèles pour effectuer la prédiction.

Nous avons décidé de faire un encodage One-Hot Encoding pour représenter chaque catégorie de manière binaire (0 ou 1), ce qui permet de conserver la distinction entre les différentes catégories tout en évitant de leur attribuer une hiérarchie implicite.

In [2]:
function encode(data, column)
    for c in unique(data[!, column])
        data[!, Symbol(c)] = ifelse.(data[!, column] .== c, 1, 0)
    end
    return data
end

function encode_data(data)
    encoded_data = deepcopy(data)
    encoded_data = encode(encoded_data, :general_type)
    encoded_data = encode(encoded_data, :transmission)
    encoded_data = encode(encoded_data, :boite)
    return encoded_data
end

function removeRows(data)
    return select!(data, Not([:type, :transmission, :boite, :general_type,]))
end

removeRows (generic function with 1 method)

In [ ]:
trainDataExplorationEncoded = encode_data(trainDataExploration)
removeRows(trainDataExplorationEncoded)
first(trainDataExplorationEncoded, 5)

### Analyse de colinéarité

La colinéarité survient lorsque deux ou plusieurs variables sont fortement corrélées, ce qui peut causer des problèmes lors de l'ajustement de modèles statistiques ou d'apprentissage automatique. Plus précisément, une forte colinéarité peut :
- Rendre difficile l'interprétation des coefficients dans un modèle.
- Affecter la stabilité numérique des algorithmes de régression.

C'est pourquoi cette analyse est essentiel pour la précision de notre modèle. Pour identifier les variables colinéaires, nous calculons une matrice de corrélation pour les variables numériques et repérons les paires dont la corrélation absolue dépasse un certain seuil (ici 0,8). Ceci nous guidera dans nos décisions futures en face aux variables explicatives.

In [ ]:
function evaluer_colinearite(data, seuil)
    numerical_data = select(data, findall(x -> eltype(x) <: Number, eachcol(data)))
    
    correlation_matrix = cor(Matrix(numerical_data))
    
    variables_colineaires = []
    valeur_R2 = []
    
    for i in 1:size(correlation_matrix, 1)
        for j in (i + 1):size(correlation_matrix, 2)
            R = abs(correlation_matrix[i, j])
            if R >= seuil
                push!(variables_colineaires, (names(numerical_data)[i], names(numerical_data)[j]))
                push!(valeur_R2, R^2)
            end
        end
    end
    
    return correlation_matrix, variables_colineaires, valeur_R2
end

In [ ]:
function calculate_vif(r_squared)
    vif = (1 / (1 - r_squared))
    return vif
end

In [ ]:
correlation_matrix, variables_colineaires, valeur_R2 = evaluer_colinearite(trainDataExplorationEncoded, 0.8)

println("Matrice de corrélation :")
println(correlation_matrix)

println("\nPaires de variables susceptibles d'être colinéaires et leurs R² :")
for (paire, r2) in zip(variables_colineaires, valeur_R2)
    vif = calculate_vif(r2)
    println("Paire : $paire, VIF : $vif")
end

L'analyse de colinéarité réalisée ci-dessus sur les données a permis de détecter plusieurs paires de variables susceptibles d’être fortement corrélées, notamment `nombre_cylindres`, `cylindree` et `volume_gaz`.En effet, le calcul des VIF confirme une très forte colinéarité sur ces trois paramètres (VIF>=10).  Ce qu’on en retire est que ces variables pourraient introduire de la redondance dans les modèles prédictifs, ce qui peut affecter la stabilité et l’interprétabilité des résultats. Étant donné que la variable d'intérêt est la consommation, il serait pertinent de considérer la suppression de certaines de ces variables colinéaires afin de simplifier le modèle et éviter les problèmes liés à la multicolinéarité.

### Données uniques

Nous avons remarqué que, parmi les 396 données, de nombreuses possèdent les mêmes caractéristiques, c'est-à-dire que leurs variables explicatives sont identiques, à l’exception de l’année. Comme mentionné dans l’analyse des graphiques de la section "Recherche de relations entre les variables explicatives et la variable d’intérêt", nous n'avons pas observé de relation entre la consommation et l'année. Ainsi, nous sélectionnons toutes les colonnes de l’ensemble d’entraînement, sauf `année` et `consommation`, afin de regrouper les données ayant des caractéristiques similaires. L’objectif est ensuite de calculer la moyenne de la consommation pour chaque groupe ainsi formé, puis de former un ensemble ayant que les données uniques. 

Il semble logique d'opter pour la moyenne dans ce cas, pusique pour des X identiques avec des valeurs de y différentes, si notre objectif est de minimiser le RMSE:

$RMSE = \sqrt{\sum_{i = 1}^{n}{(y_i - \hat{y_i})^2}} = ||{y - \hat{y}}||$

si on veut minismiser cette quantié pour une constante $\hat{y}$ tel que $RMSE$ est minimisé

$\frac{\partial}{\partial\hat{y}} \sqrt{\sum_{i = 1}^{n}{(y_i - \hat{y})^2}} = 0$

par la loi de différentiation en chaine:

$ \sqrt{ \frac{\partial}{\partial\hat{y}} \sum_{i = 1}^{n}{(y_i - \hat{y})^2}} * \frac{1}{2 * \sum_{i = 1}^{n}{(y_i - \hat{y})^2}} = 0$

ou bien

$ \sqrt{ \frac{\partial}{\partial\hat{y}} \sum_{i = 1}^{n}{(y_i^2 + \hat{y}^2 - 2 * y_i * \hat{y})}} = 0 $

$ \sqrt{ \frac{\partial}{\partial\hat{y}}( n\hat{y}^2 +  \sum_{i = 1}^{n}{y_i^2} - \sum_{i = 1}^{n}2y_i\hat{y})} = 0 $

$ \sqrt{ 2n\hat{y}  - \sum_{i = 1}^{n}2y_i} = 0 $

$2n\hat{y} = 2\sum_{i = 1}^{n}y_i$

$\hat{y} = \frac{\sum_{i = 1}^{n}y_i}{n} = \bar{y}$

La moyenne est donc l'estimateur qui minimise le RMSE pour nos observations X identiques

In [3]:
function get_unique_data(data)
    data_without_consommation = select(data, Not(:consommation, :annee))


    unique_data = combine(groupby(data, names(data_without_consommation)), :consommation => mean)
    rename!(unique_data, :consommation_mean => :consommation)
    return unique_data
end

get_unique_data (generic function with 1 method)

In [ ]:
uniqueTrainDataExploration = get_unique_data(trainDataExploration)

On peut observer que nous avons maintenant 145 données.

Examinons les relations entre la consommation et les variables explicatives pour déterminer si des changements significatifs sont observés par rapport aux graphiques présentés dans la section "Recherche de relations entre les variables explicatives et la variable d’intérêt".

In [ ]:
variables = [:type, :nombre_cylindres, :cylindree, :transmission, :boite, :volume_gaz]

plots = [
    Gadfly.plot(
        uniqueTrainDataExploration,
        x=var,
        y=:consommation,
        Geom.point,
        Guide.xlabel(string(var)),
        Guide.ylabel("consommation")
    ) for var in variables
]

set_default_plot_size(35cm, 35cm)
p = reshape(plots, (3,2))
gridstack(p)

Les graphiques ci-dessus présentent une allure similaire à ceux de la section "Recherche de relations entre les variables explicatives et la variable d’intérêt". Cela dit, il pourrait être intéressant de tester avec les modèles en ne conservant que les données uniques. Cette approche permet de simplifier le dataset en réduisant la redondance des observations ayant les mêmes caractéristiques explicatives. Cela pourrait faciliter l’apprentissage du modèle tout en améliorant son efficacité et en limitant les risques d'erreurs liés à des consommations différentes pour des données identiques.

### Analyse des données aberrantes

Pour l'analyse des données aberrantes, nous avons commencé par faire la ségrégation des données abberrantes à l'aide de la méthode classique des quantiles sur le graphique de la `cylindree` en fonction de la `consommation`.

In [ ]:
function compute_outliers(column)
    Q1 = quantile(column, 0.25)
    Q3 = quantile(column, 0.75) 
    IQR = Q3 - Q1                
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    println("Column bounds -> Lower: ", lower_bound, ", Upper: ", upper_bound, ", Q1: ", Q1, ", Q3: ", Q3, ", IQR: ", IQR)
    return findall(x -> x < lower_bound || x > upper_bound, column)
end

In [ ]:
function get_outliers_ind_quantile(data, x_col, y_col)
    x = data[!, x_col]
    y = data[!, y_col]

    outlier_indices_x = compute_outliers(x)
    outlier_indices_y = compute_outliers(y)

    combined_outlier_indices = vcat(outlier_indices_x, outlier_indices_y)
    return combined_outlier_indices
end

get_outliers_ind_quantile (generic function with 1 method)

In [ ]:
function get_outliers(data, outliers_indices)
    outliers = data[outliers_indices, :]
    return outliers
end;

Cette méthode va visualiser les données abberrantes (points noirs) et nous allons voir que ces résultas sont plutot mélancolique.

In [ ]:
function plot_outliers_quantile(uniqueD)
    outliers_indices = get_outliers_ind_quantile(uniqueD, :cylindree, :consommation)
    outliers_quantile = get_outliers(uniqueD, outliers_indices)
   
    layer_original = layer(x=uniqueD.cylindree, y=uniqueD.consommation, color=uniqueD.type)
    layer_quantile_outliers = layer(x=outliers_quantile.cylindree, y=outliers_quantile.consommation, Theme(default_color="black"))
    set_default_plot_size(15cm, 15cm)
    
    display(plot(layer_quantile_outliers, layer_original, Guide.xlabel("Cylindree"), Guide.ylabel("Consommation"), Guide.title("Original Data with Outliers")))
end

plot_outliers_quantile (generic function with 1 method)

In [ ]:
plot_outliers_quantile(uniqueTrainDataExploration)

Les résultats étant trop stricte sur les types plus consommateurs, nous avons plutôt opté pour une régression linéaire avec un seuil choisi. Ce choix de régression linéaire a été fait, car comme vu ci-dessus, suit une tendance linéaire. On commence par trouver les indices (lignes) des données aberrantes avec la méthode get_outliers_ind_regression_lin et linear_regression:

In [ ]:
function get_outliers_ind_regression_lin(data, x_col, y_col; threshold=2.5)
    slope, intercept = linear_regression(data, x_col, y_col)
    
    x = data[!, x_col]
    y = data[!, y_col]
    
    y_pred = slope .* x .+ intercept
    
    residuals = abs.(y .- y_pred)
    
    residuals_std = std(residuals)
    
    outlier_indices = findall(residuals .> threshold * residuals_std)
    
    outliers = data[outlier_indices, :]
    
    return outlier_indices
end

In [ ]:
function linear_regression(data, x_col, y_col)
    x = data[!, x_col]
    y = data[!, y_col]
    
    n = length(x)
    if n == 0
        error("Cannot compute linear regression with zero elements.")
    end
    
    x_mean = mean(x)
    y_mean = mean(y)
    
    slope = sum((x .- x_mean) .* (y .- y_mean)) / sum((x .- x_mean).^2)
    intercept = y_mean - slope * x_mean
    
    return slope, intercept
end

De plus, une méthode pour les séparer des données régulières (remove_outliers_regression_lin):

In [ ]:
function remove_outliers_regression_lin(data::DataFrame, x_col::Symbol, y_col::Symbol; threshold=2.5)
    outlier_indices = get_outliers_ind_regression_lin(data, x_col, y_col, threshold=threshold)

    keep_mask = trues(nrow(data))
    
    if !isempty(outlier_indices)
        keep_mask[outlier_indices] .= false
    end

    cleaned_data = data[keep_mask, :]
    
    return cleaned_data
end

Dernièrement, une méthode pour visualiser les données avant (points noirs) et après le retrait des données aberrantes (plot_outliers):

In [ ]:

function plot_outliers(uniqueD)
    outliers_indices = get_outliers_ind_regression_lin(uniqueD, :cylindree, :consommation)
    outliers_regression = get_outliers(uniqueD, outliers_indices)
   
    slope, intercept = linear_regression(uniqueD, :cylindree, :consommation)
    regression_line_y = slope .* uniqueD.cylindree .+ intercept
   
    layer_original = layer(x=uniqueD.cylindree, y=uniqueD.consommation, color=uniqueD.type)
    layer_linear_regression = layer(x=uniqueD.cylindree, y=regression_line_y, Geom.line, Theme(default_color="green"))
    layer_regression_outliers = layer(x=outliers_regression.cylindree, y=outliers_regression.consommation, Geom.point, Theme(default_color="black"))
    set_default_plot_size(15cm, 15cm)
    
    display(plot(layer_regression_outliers, layer_original, layer_linear_regression, Guide.xlabel("Cylindree"), Guide.ylabel("Consommation"), Guide.title("Original Data with Outliers")))
   
    cleaned_data = remove_outliers_regression_lin(uniqueD, :cylindree, :consommation, threshold=2.5)
   
    slope_cleaned, intercept_cleaned = linear_regression(cleaned_data, :cylindree, :consommation)
    regression_line_y_cleaned = slope_cleaned .* cleaned_data.cylindree .+ intercept_cleaned
   
   layer_cleaned = layer(x=cleaned_data.cylindree, y=cleaned_data.consommation, color=cleaned_data.type, Theme(default_color="blue"))
   layer_linear_regression_cleaned = layer(x=cleaned_data.cylindree, y=regression_line_y_cleaned, Geom.line, Theme(default_color="green"))
   
   display(plot(layer_cleaned, layer_linear_regression_cleaned, 
                Guide.xlabel("Cylindree"), Guide.ylabel("Consommation"), Guide.title("Cleaned Data with Regression Line")))
   
end

In [ ]:
plot_outliers(uniqueTrainDataExploration)

Cette analyse et ségrégation des données abberrantes pourront nous être utile plus tard dans nos recherches.

## Choix variables

Suite à l'exploration des données, nous allons construire un DataFrame avec toutes les variables explicatives que nous avons évaluer comme étant pertinente pour la prédiction de la consommation....

### Standardisation des données

Nous avons standardisé les données numériques pour mettre toutes nos données sur la même échelle. De cette façon, on peut comparer leur importance relative. De plus, une variable ayant des valeurs très hautes comparées aux autres n'influencera pas les données de façon plus significative qu'elle le devrait.

In [6]:
COMSOMMATION_MEAN = mean(trainData.consommation)
COMSOMMATION_STD = std(trainData.consommation)

2.139763088813657

In [7]:
function standardize(data)
    return (data .- mean(data)) ./ std(data)
end

function standardize_data(data)
    stddata = deepcopy(data)
   for col in names(stddata)
        if eltype(stddata[!, col]) <: Number && col != "id"
            stddata[!, col] = standardize(stddata[!, col])
        end
    end
    return stddata
end

standardize_data (generic function with 1 method)

In [8]:
function add_rows(data)
    data[!,:volume_gaz] = data[!,:nombre_cylindres] .* data[!,:cylindree]

    #https://www.insurancenavy.com/average-car-weight/
    #https://www.auto-tests.com/fr/lightest-weight/Wagon/all/
    weight_dict = Dict("voiture_moyenne" => 3300, "VUS_petit" => 3500, "voiture_compacte" => 2800, "voiture_deux_places" => 2800, "voiture_minicompacte" => 1500, "VUS_standard" => 5000, "monospace" => 4500, "voiture_sous_compacte" => 2600, "camionnette_petit" => 4200, "break_petit" => 2640, "voiture_grande" => 4400, "camionnette_standard" => 4700, "break_moyen" => 3300)
    data[!, :weight] = [weight_dict[t] for t in data[!, :type]]

    general_type_dict = Dict("voiture_moyenne" => "voiture", "VUS_petit" => "VUS", "voiture_compacte" => "voiture", "voiture_deux_places" => "voiture", "voiture_minicompacte" => "voiture", "VUS_standard" => "VUS", "monospace" => "camionnette", "voiture_sous_compacte" => "voiture", "camionnette_petit" => "camionnette", "break_petit" => "break", "voiture_grande" => "voiture", "camionnette_standard" => "camionnette", "break_moyen" => "break")
    data[!, :general_type] = [general_type_dict[t] for t in data[!, :type]]
    
    return data
end

add_rows (generic function with 1 method)

In [9]:
function getStandardEncodedData(data)
    data_copy = deepcopy(data)
    standardised_data = get_unique_data(data_copy)
    standardised_data = add_rows(data_copy)
    standardised_data = standardize_data(data_copy)
    standardised_data = encode_data(standardised_data)
    standardised_data = removeRows(standardised_data)
    
    return standardised_data
end

getStandardEncodedData (generic function with 1 method)

## Évaluation des modèles

### calcul rmse

Nous avons évalué nos modèles avec la métrique rmse, car c'est celle-ci qui est utilisée pour évaluer nos prédictions dans le concours.

In [10]:
function rmse(y, prediction)
    return sqrt(mean((prediction .- y).^2))
end

rmse (generic function with 1 method)

Nous avons calculé le rmse moyen en faisant une séparation aléatoire des données à chaque fois afin d'avoir une idée générale du rmse de chaque modèle.

In [11]:
function evaluate_rmse(data, model, nrange = 1000, test_size = 0.2)
    n = 0
    for i in range(0, 1, length=nrange)
        train_data, test_data = train_test_split(data, test_size)
        n += model(train_data, test_data)[1]
    end
    average_rmse = n/nrange
    print("average rmse: ", average_rmse, "\n")
    return average_rmse
end

evaluate_rmse (generic function with 3 methods)

### Séparation des ensemble d'entrainement et de validation

In [12]:
function train_test_split(data, test_size=0.2, shuffle=true)
    n = size(data, 1)
    test_size = floor(Int, n * test_size)
    
    if shuffle
        indices = randperm(n)
    else
        indices = 1:n
    end
    
    test_indices = indices[1:test_size]
    train_indices = indices[test_size+1:end]
    
    train_data = data[train_indices, :]
    test_data = data[test_indices, :]
    
    return train_data, test_data
end

train_test_split (generic function with 3 methods)

### Les différents modèles

Voici une liste des toutes les modèles prédictifs que nous avons implémenter et tester afin d'obtenir les meilleures prédictions de consommation d'essence:
- Régression
- Régression ridge
- Régression lasso
- Régression polynomiale
- Régression SVD
- Arbres de classification
- Autres?

Voici leurs implémentations:

#### Régression linéaire

In [13]:
function regression(training_data, test_data = nothing)	
    X_train =  Matrix(training_data[:, Not(:consommation, :id)])
    y_train = training_data[:, :consommation]

    beta = X_train \ y_train

    rmseval = 0.0
    if test_data != nothing
        X_test = Matrix(test_data[:, Not(:consommation, :id)])
        y_test = test_data[:, :consommation]
        y_predict =  X_test * beta
        y_predict = (y_predict .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        rmseval = rmse(y_test, y_predict)
    end
    
    return rmseval, beta
end

regression (generic function with 2 methods)

In [14]:
evaluate_rmse(getStandardEncodedData(trainData), regression)


average rmse: 0.9475703528621615


0.9475703528621615

#### Régression ridge

In [15]:
function ridge_regression(training_data, test_data = nothing, lambda=0.5)
    X_train = Matrix(training_data[:, Not([:consommation, :id])])
    y_train = training_data[:, :consommation]
    beta = (X_train'X_train + lambda*I)\X_train'y_train

    rmseval = 0.0
    if test_data != nothing
        X_test = Matrix(test_data[:, Not(:consommation, :id)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        rmseval = rmse(y_test, ychap)
    end
    return rmseval, beta
end

ridge_regression (generic function with 3 methods)

In [16]:
evaluate_rmse(getStandardEncodedData(trainData), ridge_regression)

average rmse: 0.9565591365482474


0.9565591365482474

#### Régression SVD

In [17]:
function svd_regression(training_data, test_data = nothing)
    X_train = Matrix(training_data[:, Not([:consommation, :id])])
    y_train = training_data[:, :consommation]
    
    U, S, V = svd(X_train)

    beta = V' * Diagonal([s > 1e-10 ? 1/s : 0 for s in S]) * U' * y_train

    rmseval = 0.0
    if test_data != nothing
        X_test = Matrix(test_data[:, Not(:consommation,:id)])
        y_test = test_data[:, :consommation]
        ychap =  X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        rmseval = rmse(y_test, ychap)
    end
    return rmseval, beta
end

svd_regression (generic function with 2 methods)

In [18]:
evaluate_rmse(getStandardEncodedData(trainData), svd_regression)

2.466190563236575

average rmse: 2.466190563236575


#### régression polynomiale

In [19]:
function construct_structure(x::Matrix{<:Real}, order::Int)
    n, m = size(x)
    poly_terms = [x[:, j].^p for j in 1:m, p in 0:order]
    X = hcat(poly_terms...)
    return X
end

function polynomial_regression(training_data, test_data = nothing, degree = 3)
    X_train = construct_structure(Matrix(training_data[:, Not([:consommation, :id])]), degree)
    y_train = training_data[:, :consommation]

    beta = X_train \ y_train

    rmseval = 0.0

    if test_data != nothing
        X_test = construct_structure(Matrix(test_data[:, Not([:consommation, :id])]), degree)
        y_test = test_data[:, :consommation]
        ychap = X_test * beta
        ychap = (ychap .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN
        y_test = (y_test .* COMSOMMATION_STD) .+ COMSOMMATION_MEAN

        rmseval = rmse(y_test, ychap)
    end

    return rmseval, beta
end

polynomial_regression (generic function with 3 methods)

In [20]:
evaluate_rmse(getStandardEncodedData(trainData), polynomial_regression)

average rmse: 0.9120975660802773


0.9120975660802773

#### Comparaison des modèles

Suite à plusieurs tests de chacune d'entrée elle le modèles ayant proposer les meilleures prédictions est ... parce que ... À compléter

## Conclusion et amélioration